# Выбор локации для скважины

Задача проекта: решить, где бурить новую скважину для нефтедобывающей компании.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищем месторождения, для каждого определяем значения признаков;
- Строим модель и оцениваем объём запасов;
- Выбираем месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

Нам предоставлены следующие данные:
- `id` — уникальный идентификатор скважины;
- `f0, f1, f2` — три признака точек (обозначение признаков неизвестно, но сами признаки значимы);
- `product` — объём запасов в скважине (тыс. баррелей).

Для начала загрузим и изучим данные.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#настройка формата отображения чисел
pd.options.display.float_format = '{:,.2f}'.format

%matplotlib inline

Данные геологоразведки трёх регионов находятся в файлах:

In [2]:
geo_data_0 = pd.read_csv('geo_data_0.csv')
geo_data_1 = pd.read_csv('geo_data_1.csv')
geo_data_2 = pd.read_csv('geo_data_2.csv')

In [3]:
data = [geo_data_0, geo_data_1, geo_data_2]
for dataset in data:
    print(dataset.head())
    print()
print()
print()
for dataset in data:
    print(dataset.info())
    print()
print()
print()
for dataset in data:
    print(dataset.describe())
    print()

      id    f0    f1   f2  product
0  txEyH  0.71 -0.50 1.22   105.28
1  2acmU  1.33 -0.34 4.37    73.04
2  409Wp  1.02  0.15 1.42    85.27
3  iJLyR -0.03  0.14 2.98   168.62
4  Xdl7t  1.99  0.16 4.75   154.04

      id     f0     f1    f2  product
0  kBEdx -15.00  -8.28 -0.01     3.18
1  62mP7  14.27  -3.48  1.00    26.95
2  vyE1P   6.26  -5.95  5.00   134.77
3  KcrkZ -13.08 -11.51  5.00   137.95
4  AHL4O  12.70  -8.15  5.00   134.77

      id    f0    f1    f2  product
0  fwXo0 -1.15  0.96 -0.83    27.76
1  WJtFt  0.26  0.27 -2.53    56.07
2  ovLUW  0.19  0.29 -5.59    62.87
3  q6cA6  2.24 -0.55  0.93   114.57
4  WPMUX -0.52  1.72  5.90   149.60



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float6

Проанализируем распределения целевого признака в различных регионах:

In [4]:
product_destribution = pd.concat([geo_data_0['product'].describe(), geo_data_1['product'].describe(), 
                                  geo_data_2['product'].describe()], axis=1)
product_destribution.columns = ['Region 1', 'Region 2', 'Region 3']
product_destribution

,Region 1,Region 2,Region 3
count,"100,000.00","100,000.00","100,000.00"
mean,92.50,68.83,95.00
std,44.29,45.94,44.75
min,0.00,0.00,0.00
25%,56.50,26.95,59.45
50%,91.85,57.09,94.93
75%,128.56,107.81,130.60
max,185.36,137.95,190.03


Мы убедились, что основные параметры распределения целевого признака в трех регионах примерно одинаковые (при этом в среднем во втором регионе значения ниже).

## Обучение и проверка модели

### Масштабирование признаков

Далее подготовим данные для обучения модели. Все признаки у нас количественные, их необходимо масштабировать. Сделаем это с помощью стандартизации. Стобцец id отбросим, т. к. он не содержит значимой информации для модели.

In [5]:
#создадим словарь с датасетами для автоматизации их обработки
datasets = {'region_1': geo_data_0, 'region_2': geo_data_1, 'region_3': geo_data_2}

In [6]:
#разделим датасеты на фичи и целевой признак
features = {}
targets = {}

for region, dataset in datasets.items():
    features[region] = dataset.drop(columns=['id', 'product'])
    targets[region] = dataset['product']

print("Размер датасетов с фичами:", features['region_2'].shape)
print("Размер датасетов с целевым признаком:", targets['region_2'].shape)

Размер датасетов с фичами: (100000, 3)
Размер датасетов с целевым признаком: (100000,)


In [7]:
#разобьем датасеты на тренировочную и валидационную выборки

features_train = {}
features_valid = {}

for region, feature in features.items():
    features_train[region], features_valid[region] = train_test_split(
                                                        feature, random_state = 42, train_size=0.75)

    
targets_train = {}
targets_valid = {}  
    
for region, targets in targets.items():
    targets_train[region], targets_valid[region] = train_test_split(targets, random_state = 42, train_size=0.75)


print("Размер обучающих выборок и целевого признака", features_train['region_2'].shape, 
      targets_train['region_2'].shape)
print("Размер валидационных выборок и целевого признака", features_valid['region_2'].shape, 
      targets_valid['region_2'].shape)

Размер обучающих выборок и целевого признака (75000, 3) (75000,)
Размер валидационных выборок и целевого признака (25000, 3) (25000,)


In [8]:
#игнорирование предупреждения об ошибки при записи масштабированных признаков в исходный датафрейм
pd.options.mode.chained_assignment = None

#настройка скелера на обучающих датасетах и их масштабирование
scalers = {}

for region, feature_train in features_train.items():
    scalers[region] = StandardScaler()
    scalers[region].fit(feature_train)
    features_train[region] = scalers[region].transform(feature_train)
    features_train[region] = pd.DataFrame(features_train[region], columns=['f0', 'f1', 'f2'])

#масштабирование количественных признаков в валидационных выборках
for region, feature_valid in features_valid.items():
    features_valid[region] = scalers[region].transform(feature_valid)
    features_valid[region] = pd.DataFrame(features_valid[region], columns=['f0', 'f1', 'f2'])

Выведем информацию о распределении масштабированных признаков:

In [9]:
pd.concat([features_train['region_1'].describe(), features_train['region_2'].describe(), 
           features_train['region_3'].describe()], axis=1)

,f0,f1,f2,f0,f1,f2,f0,f1,f2
count,"75,000.00","75,000.00","75,000.00","75,000.00","75,000.00","75,000.00","75,000.00","75,000.00","75,000.00"
mean,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-2.19,-2.18,-3.89,-3.65,-4.21,-1.48,-5.06,-4.09,-4.16
25%,-0.66,-0.89,-0.68,-0.83,-0.68,-0.88,-0.67,-0.68,-0.68
50%,0.00,0.00,0.00,0.00,-0.00,-0.28,0.00,-0.00,-0.00
75%,0.65,0.89,0.68,0.83,0.68,0.88,0.67,0.67,0.68
max,2.14,2.15,4.15,3.16,4.21,1.48,4.18,4.53,4.10


In [10]:
pd.concat([features_valid['region_1'].describe(), features_valid['region_2'].describe(), 
           features_valid['region_3'].describe()], axis=1)

,f0,f1,f2,f0,f1,f2,f0,f1,f2
count,"25,000.00","25,000.00","25,000.00","25,000.00","25,000.00","25,000.00","25,000.00","25,000.00","25,000.00"
mean,0.00,-0.01,-0.01,0.01,0.01,-0.00,-0.00,0.01,-0.01
std,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-2.02,-2.18,-4.49,-2.89,-4.02,-1.48,-3.53,-3.87,-3.95
25%,-0.65,-0.92,-0.69,-0.82,-0.67,-0.88,-0.68,-0.67,-0.69
50%,0.00,-0.01,-0.01,-0.00,0.00,-0.29,0.00,0.00,-0.02
75%,0.67,0.90,0.67,0.84,0.68,0.88,0.67,0.68,0.67
max,2.04,2.17,3.85,3.04,4.60,1.48,4.17,4.48,3.78


Стандартное отклонение выбранных признаков равно единице, а среднее нулю. Стандартизация выполнена успешно.

### Обучение модели

Обучим модель линейной регрессии и сделаем предсказания на валидационной выборке.

In [11]:
#напишем функцию обучения модели
def model_fit(features_train, target_train):
    model = LinearRegression()
    model.fit(features_train, target_train)
    return model

#напишем функцию предсказания значений
def model_predict(features_valid, region):
    prediction = models[region].predict(features_valid)
    return prediction

In [12]:
#обучим модель и рассчитаем предсказания для трех регионов
models = {}

for (region, feature_train), (region, target_train) in zip(features_train.items(), targets_train.items()):
    models[region] = model_fit(feature_train, target_train)
    
    
predictions = {}

for region, feature_valid in features_valid.items():
    predictions[region] = model_predict(feature_valid, region)

In [13]:
#создадим датафрейм со средними значениями предсказаний и RMSE в разрезе регионов

region_names = ['region_1', 'region_2', 'region_3']
region_predictions_mean = []
region_RMSE = []
region_factual_mean = []

for region in region_names:
    
    region_predictions_mean.append(predictions[region].mean())
    
    region_RMSE.append((
                        mean_squared_error
                       (targets_valid[region], predictions[region]) ** 0.5)
                      )
    
    region_factual_mean.append(datasets[region]['product'].mean())

    
indicators = pd.DataFrame(
        {'mean_predicted': region_predictions_mean,
         'mean_factual': region_factual_mean,
         'RMSE': region_RMSE, 
        }, 
    index=region_names).transpose()

indicators

,region_1,region_2,region_3
mean_predicted,92.40,68.71,94.77
mean_factual,92.50,68.83,95.00
RMSE,37.76,0.89,40.15


In [14]:
#выведем на экран датафрейм с коэффициентами моделей для различных регионов
print('Коэффициенты признаков обученных моделей')

pd.DataFrame(
        {'region_1': models['region_1'].coef_,
        'region_2': models['region_2'].coef_,
        'region_3': models['region_3'].coef_
        },
    index=['f0', 'f1', 'f2'])

Коэффициенты признаков обученных моделей


,region_1,region_2,region_3
f0,3.34,-1.30,-0.15
f1,-7.18,-0.11,-0.03
f2,21.42,45.91,19.99


### Выводы

По итогам предсказаний модели мы можем сделать следующие выводы:
1. Средние предсказанные запасы сырья в 1-м и 3-м регионах примерно одинаковы, а во 2-м регионе существенно ниже. При этом средние значения предсказаний близки к истинным средним, полученным из исходных датасетов.


2. Значения RMSE для 1-го и 3-го регионов достаточно большие относительно средних значений. Однако, т. к. мы будем использовать предсказания модели для определения наиболее богатых месторождений (их ранжирования), нам важна относительная погрешность, а не абсолютная. Низкое значение RMSE для 2-го региона объясняется высокой корреляцией между одним из признаков (f2) и целевым признаком в датасете, что позволило модели сделать точные предсказания на основе этого признака. 


3. Во всех регионах наибольший вклад в значение целевого признака вносит признак f2.

## Подготовка к расчёту прибыли

На данном шаге определим необходимые параметры, а также напишем функцию для расчета прибыли от разработки месторождений в различных регионах. Для удобства расчеты будем проводить в тысячах рублей.

In [15]:
#определим и запишем небходимые для расчета переменные
REVENUE_FOR_THSND_BARRELS = 450
TOTAL_BUDGET = 10000000
OILFIELDS_COUNT_FOR_PRODUCTION = 200
OILFIELDS_COUNT_FOR_RESEARCH = 500
WELL_COST = TOTAL_BUDGET / OILFIELDS_COUNT_FOR_PRODUCTION

In [16]:
#рассчитаем объем сырья, необходимый для безубыточной разработки одной скважины
minimal_production = WELL_COST / REVENUE_FOR_THSND_BARRELS

print('Минимальный запас сырья для безубыточной разработки месторождения, тыс. барр.:', minimal_production)
print()

print('Средние запасы сырья в месторождениях:')
indicators.loc['mean_factual']

Минимальный запас сырья для безубыточной разработки месторождения, тыс. барр.: 111.11111111111111

Средние запасы сырья в месторождениях:


region_1   92.50
region_2   68.83
region_3   95.00
Name: mean_factual, dtype: float64

На основе расчетов мы можем сделать вывод, что средних значений запасов нефти ни в одном из трех регионов недостаточно для безубыточной разработки новых месторождений. Далее с помощью обученных моделей отберем наиболее богатые месторождения и проанализируем достаточность запасов в них.

## Расчёт прибыли и рисков 

С помощью предсказаний обученных моделей определим 200 наиболее перспективных месторождений для разработки и рассчитаем достаточность фактически разведанных в них запасов для получения прибыли.

In [17]:
#напишем функцию для расчета запасов нефти
def oil_reserve_calc(targets, predictions, region):
    
    predictions_sorted = pd.Series(predictions).sort_values(ascending=False)
    selected = (pd.Series(targets[region])
                .iloc[predictions_sorted.index]
                [:OILFIELDS_COUNT_FOR_PRODUCTION])
    return selected.sum()

In [18]:
#создадим датафрейм с рассчитанными значениями прибыли на основе наилучших месторождений для каждого региона

oil_reserves = []
profit = []

for region, prediction in predictions.items():
    
    oil_reserves_region = oil_reserve_calc(targets_valid, prediction, region)
    
    oil_reserves.append(oil_reserves_region)
    
    profit.append(oil_reserves_region * REVENUE_FOR_THSND_BARRELS - TOTAL_BUDGET)
    
pd.DataFrame(
        {'oil_reserves': oil_reserves,
         'profit': profit
        }, 
    index=region_names).transpose()

,region_1,region_2,region_3
oil_reserves,"29,686.98","27,589.08","27,996.83"
profit,"3,359,141.11","2,415,086.70","2,598,571.76"


При отборе 200 лучших месторождений в каждом из регионов прибыль получилась положительной. 

При этом запасы нефти и прибыль во втором регионе несущественно меньше, чем в третьем, несмотря на большую разницу в средних значениях запаса нефти, выявленную на предыдующих этапах исследования. Это можно объяснить более высокой концентрацией запасов в самых богатых месторождениях второго региона по сравнению с третьим.

Далее посчитаем риски и прибыль для каждого региона. Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [19]:
#напишем функцию для расчета прибыли для применения в бутстрепе
def revenue(target, predictions):
    
    predictions = pd.Series(predictions)
    predictions_sorted = predictions.sort_values(ascending=False)
    selected = target[predictions_sorted.index][:OILFIELDS_COUNT_FOR_PRODUCTION]
    profit = selected.sum() * REVENUE_FOR_THSND_BARRELS - TOTAL_BUDGET
    return profit

In [20]:
state = np.random.RandomState(42)
    

#бутстреп с 1000 повторений, где 500 - кол-во анализируемых месторождений, 
# 200 - кол-во месторождений, планируемых к разработке
def bootstrap (target, predictions):
    
    values = []
    
    for i in range(1000):
        target = pd.Series(target).reset_index(drop=True)
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        predictions_subsample = pd.Series(predictions)[target_subsample.index]
        
        rev = revenue(target_subsample, predictions_subsample)
        values.append(rev)
        
    values = pd.Series(values)
    
    profit_mean = values.mean()
    
    confidence_interval = (values.quantile(0.025), values.quantile(0.975))
        
    count = 0
    for i in range(len(values)):
        if values[i] < 0:
            count += 1
        
    loss_risk_percent = count / len(values) * 100
    
    return profit_mean, confidence_interval, loss_risk_percent

In [21]:
profit_mean_region_1, confidence_interval, loss_risk_percent  = bootstrap(
    targets_valid['region_1'], predictions['region_1'])

In [22]:
profit_mean = {}
confidence_intervals = {}
loss_risk_percent = {}

for (region, target), (region, prediction) in zip(targets_valid.items(), predictions.items()):
    profit_mean[region], confidence_intervals[region], loss_risk_percent[region] = bootstrap(target, prediction)

In [23]:
final_results = pd.DataFrame.from_dict([profit_mean, confidence_intervals, loss_risk_percent])
final_results.index=(['profit_mean', 'confidence_intervals', 'loss_risk_percent'])
final_results

,region_1,region_2,region_3
profit_mean,"450,083.47","501,333.92","394,436.69"
confidence_intervals,"(-44486.92448507527, 972499.6712774887)","(72401.51090591788, 919399.9638429123)","(-151950.95935870882, 901488.1437388246)"
loss_risk_percent,4.50,0.60,8.30


### Вывод

По условиям бизнес-задачи для разработки подходят только регионы с риском убытков менее 2.5%. По итогам применения техники бутстрепа единственным таким регионом является регион №2 (риск убытков 0.6%). Также это регион с наибольшей средней прибылью (501.3 млн руб.). С вероятностью 95% прибыль от разработки месторождений в данном регионе будет находиться в интервале от 72.4 до 919.4 млн руб.

При этом при первоначальном анализе средние запасы нефти в месторождениях второго региона были ниже, чем в 1-м и 3-м регионах. Разница между результатами первоначального анализа и итоговыми выводами на основе данных бутстрепа обусловлена распределением запасов в данном регионе (более равномерное, чем в 1-м и 3-м).